# Data parsing 

This are the steps to generate useable data from any github project. 

1. Requirements
    - CSV: 
        CSV datasheet of the history of the repository. 
    - JSON:
        Json of the history of the repository. 

2. Steps to produce and what the outputs are:
    | Step | Description | Input | Output |
    | --- | --- | --- | --- |
    | 1 | Clone the repository |  |  |
    | 2 | Get a list of all the developers associated with the project | CSV (data sheet of the history of the repository) | CSV |
    | 3 | Get developers experience over the years since the creation date of the project. | CSV (data sheet of the history of the repository), JSON (the history of the repository) | CSV |
    | 4 | Get developers authering of a task over the years since the creation date of the project. | CSV (data sheet of the history of the repository), JSON (the history of the repository) | CSV |
    | 5 | Get developers commit or provisioning of a task over the years since the creation date of the project. | CSV (data sheet of the history of the repository), JSON (the history of the repository) | CSV |
    | 6 |  Getting task information | JSON (the history of the repository), CSV (getting the output of developers experience (2nd Step)) | CSV |
    | 7 | Setting task information | CSV (getting the output of tasks (5th Step)) | CSV |
    | 8 | Add actual metrics calculation and Clean the data so that it can be used for causal estimations | CSV (getting the output of tasks (6th Step)) and removing columns, replacing strings with numbers... etc. | CSV |
    
    <hr>

## Steps to reproduce

### Imports

In [4]:
import csv
import json
import os
import re
import pandas as pd
import datetime
from dateutil import parser
import spacy
import subprocess
from contribution_complexity.compute import find_commits_for_issue
from contribution_complexity.metrics import compute_contrib_compl


nlp = spacy.load("en_core_web_sm")

### Helper functions setup

In [5]:
def read_json_file(filename):
    """Reads a JSON file and returns the data as a Python object."""
    with open(filename, 'r') as jsonfile:
        data = json.load(jsonfile)
    return data

def read_csv_file(filename):
    """Reads a CSV file and returns the data as a list of lists."""
    with open(filename, 'r') as csvfile:
        csvreader = csv.reader(csvfile)
        data = [row for row in csvreader]
    return data

### Add the name of the project for which you would like to generate a dataset.

In [35]:
name = 'vue'

## Step 1

### Clone the source repository


### Set up the paths to the CSV and JSON files

In [9]:
save_path = 'data/useable/'
csv_path = 'data/' + name + '/' + name + '.csv'  # file_utils.choose_file()
json_path = 'data/' + name + '/' + name + '.json'  # file_utils.choose_file()
save_path = save_path + name + '/'

# Load Json from the path and convert it
json_obj = read_json_file(json_path)

# Set up the paths to the tasks CSV and tasks_output
tasks_path = save_path + "tasks.csv"
tasks_output_path = "tasks_output.csv"


# Load csv from the path and convert it to a list of lists
csv_lists = read_csv_file(csv_path)

# Set path of the source code
path_to_repo = "./tmp/"+name+"/"

## Step 2
### Analyze the data using the appropriate analysis module

In [10]:
def analyze_data_devs_by_index(input_json_path, output_csv_path):
 
    # load the JSON file
    with open(input_json_path, 'r') as f:
        data = json.load(f)

    # extract the unique email addresses of the authors and committers
    emails_set = set()
    for commit in data['commits']:
        emails_set.add(commit['authored_by']['author_email'])
        emails_set.add(commit['committed_by']['committer_email'])

    # write out the unique values to the output CSV file
    with open(output_csv_path, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        # write the header row
        csvwriter.writerow(['index', 'developer'])
        # write each unique value as a row in the output CSV file
        for i, email in enumerate(emails_set):
            csvwriter.writerow([i, email])


    output_file_name = os.path.basename(output_csv_path)

    return emails_set

Call the function to create the file

In [11]:
csv_analysis_results = analyze_data_devs_by_index(
    json_path, save_path+"developers.csv")

## Step 3

### Generating a csv of developers that authored within the years of the project


In [33]:
def get_list_of_years(input_list):
    # create an empty set to store unique values

    years = set()
    for row in input_list[1:]:
        a_at = parser.parse(row[5]).strftime('%m/%d/%Y, %H:%M:%S')
        c_at = parser.parse(row[7]).strftime('%m/%d/%Y, %H:%M:%S')
        authored_at = datetime.datetime.strptime(a_at, '%m/%d/%Y, %H:%M:%S')
        committed_at = datetime.datetime.strptime(c_at, '%m/%d/%Y, %H:%M:%S')
        years.add(authored_at.year)
        years.add(committed_at.year)

    list_of_years = sorted(list(years))
    return list_of_years


def get_year_from_date(date_string):
    date = datetime.datetime.strptime(date_string, '%m/%d/%Y, %H:%M:%S')
    return date.year

def generate_csv_devs_authored_the_years(input_csv_list, json_obj, output):
    # write out the unique values to the output CSV file
    with open(output, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        list_of_years = get_list_of_years(input_csv_list)
        # write the header row
        header_row = list(list_of_years)
        header_row.insert(0, 'id')
        header_row.append('total_authored')
        csvwriter.writerow(header_row)

        # get the set of unique developers
        devs = set()
        for commit in json_obj["commits"]:
            if 'authored_by' in commit and 'author_email' in commit['authored_by']:
                dev = commit['authored_by']['author_email']
                devs.add(dev)

        # create a dictionary to store the number of commits made by each developer in each year
        authors_by_year_by_dev = {
            dev: {str(year): 0 for year in list_of_years} for dev in devs}

        # iterate through the rows of the CSV file and count the number of commits made by each developer in each year
        for commit in json_obj["commits"]:
            if 'authored_by' in commit and 'author_email' in commit['authored_by']:
                year = get_year_from_date(commit['authored_at'])
                dev = commit['authored_by']['author_email']
                authors_by_year_by_dev[dev][str(year)] += 1

        # write out the data rows to the CSV file
        for dev in sorted(devs):
            data_row = [dev]
            total_authored = 0
            for year in list_of_years:
                data_row.append(authors_by_year_by_dev[dev][str(year)])
                total_authored += authors_by_year_by_dev[dev][str(year)]
            data_row.append(total_authored)
            csvwriter.writerow(data_row)
        
        print(f"The list of unique author values are {len(list(devs))}.")


Call the function to create the file

In [16]:
generate_csv_devs_authored_the_years(
    csv_lists, json_obj, save_path + "dev_author_experience.csv")

The list of unique author values are 290.


## Step 4
### Generating a csv of developers that committed within the years of the project


In [17]:
def generate_csv_devs_commit_the_years(input_csv_list, json_obj, output):
    # write out the unique values to the output CSV file
    with open(output, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        list_of_years = get_list_of_years(input_csv_list)
        # write the header row
        header_row = list(list_of_years)
        header_row.insert(0, 'id')
        header_row.append('total_commits')
        csvwriter.writerow(header_row)

        # get the set of unique developers
        devs = set()
        for commit in json_obj["commits"]:
            if 'committed_by' in commit and 'committer_email' in commit['committed_by']:
                dev = commit['committed_by']['committer_email']
                devs.add(dev)

        # create a dictionary to store the number of commits made by each developer in each year
        committers_by_year_by_dev = {
            dev: {str(year): 0 for year in list_of_years} for dev in devs}

        # iterate through the rows of the CSV file and count the number of commits made by each developer in each year
        for commit in json_obj["commits"]:
            if 'committed_by' in commit and 'committer_email' in commit['committed_by']:
                year = get_year_from_date(commit['commited_at'])
                dev = commit['committed_by']['committer_email']
                committers_by_year_by_dev[dev][str(year)] += 1


        # write out the data rows to the CSV file
        for dev in sorted(devs):
            data_row = [dev]
            total_committed = 0
            for year in list_of_years:
                data_row.append(committers_by_year_by_dev[dev][str(year)])
                total_committed += committers_by_year_by_dev[dev][str(year)]
            data_row.append(total_committed)
            csvwriter.writerow(data_row)

        print(f"The list of unique commit values are {len(list(devs))}.")

Call the function to create the file

In [18]:
generate_csv_devs_commit_the_years(
    csv_lists, json_obj, save_path + "dev_commit_experience.csv")

The list of unique commit values are 12.


## Step 5

### Generating a csv of all developers that committed and authored within the years of the project to and getting their experience with the source code

In [21]:
def generate_csv_devs_experience_the_years(input_csv_list, json_obj, output):

    with open(output, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)

        # get the list of years from the input CSV file
        list_of_years = get_list_of_years(input_csv_list)

        # write the header row to the output CSV file
        header_row = list(list_of_years)
        header_row.insert(0, 'id')
        csvwriter.writerow(header_row)

        # get the set of unique developers from the input JSON file
        devs = set()
        for commit in json_obj["commits"]:
            if 'authored_by' in commit and 'author_email' in commit['authored_by']:
                dev = commit['authored_by']['author_email']
                devs.add(dev)
            if 'committed_by' in commit and 'committer_email' in commit['committed_by']:
                dev = commit['committed_by']['committer_email']
                devs.add(dev)

        # create a dictionary to store the number of commits made by each developer in each year
        active_by_year_by_dev = {
            dev: {str(year): -1 for year in list_of_years} for dev in devs}
       
        # iterate through the rows of the input JSON file and count the number of commits made by each developer in each year
        for commit in json_obj["commits"]:
            if 'authored_by' in commit and 'author_email' in commit['authored_by']:
                year = get_year_from_date(commit['authored_at'])
                dev = commit['authored_by']['author_email']
                if(active_by_year_by_dev[dev][str(year)] == -1):
                    active_by_year_by_dev[dev][str(year)] = 0
                else:
                    active_by_year_by_dev[dev][str(year)] = 1

            if 'committed_by' in commit and 'committer_email' in commit['committed_by']:
                year = get_year_from_date(commit['commited_at'])
                dev = commit['committed_by']['committer_email']
                if(active_by_year_by_dev[dev][str(year)] == -1):
                    active_by_year_by_dev[dev][str(year)] = 0
                else:
                    active_by_year_by_dev[dev][str(year)] = 1

        # calculate the years of experience for each developer
        for dev in devs:
            counter = 0
            for year in list_of_years:
                if active_by_year_by_dev[dev][str(year)] > -1:
                    active_by_year_by_dev[dev][str(year)] = counter
                    counter += 1

        # write out the data rows to the output CSV file
        for dev in sorted(devs):
            data_row = [dev]
            for year in list_of_years:
                data_row.append(active_by_year_by_dev[dev][str(year)])
            csvwriter.writerow(data_row)
        print(f"The list of unique values are {len(list(devs))}.")
        
        

Call the function to create the file

In [22]:
generate_csv_devs_experience_the_years(
    csv_lists, json_obj, save_path + "dev_experience.csv")

The list of unique values are 292.


## Step 6
### Getting task information

#### Helper tools

In [30]:
# Define keywords and phrases for each category
feature_keywords = ["feature", "add", "new", "feature", "functionality",
                    "enhancement", "improvement", "capability", "option"]
bug_keywords = ["fix", "error", "bug", "issue",
                "problem", "crash", "fault", "flaw"]
refactoring_keywords = ["refactor", "improve", "clean", "optimize",
                        "simplify", "streamline", "consolidate", "reorganize"]
documentation_keywords = ["document", "explain", "instruction",
                          "guide", "manual", "tutorial", "reference", "help"]
testing_keywords = ["test", "unit", "integration", "automated",
                    "manual", "coverage", "regression", "performance"]
deployment_keywords = ["deploy", "release", "ship",
                       "install", "configure", "setup", "package", "deliver"]
maintenance_keywords = ["maintain", "support", "upgrade",
                        "update", "monitor", "backup", "restore", "troubleshoot"]

categories = ["Feature requests or new feature", "Bug fixes",
              "Code Refactoring", "Documentation", "Testing", "Deployment", "Maintenance"]

def get_task_info(message, is_bug, is_ref):
    category_counts = {category: 0 for category in categories}
    doc = nlp(message.lower())

    for token in doc:
        if any(keyword in token.text for keyword in feature_keywords):
            category_counts["Feature requests or new feature"] += 1
        if any(keyword in token.text for keyword in bug_keywords):
            category_counts["Bug fixes"] += 1
        if any(keyword in token.text for keyword in refactoring_keywords):
            category_counts["Code Refactoring"] += 1
        if any(keyword in token.text for keyword in documentation_keywords):
            category_counts["Documentation"] += 1
        if any(keyword in token.text for keyword in testing_keywords):
            category_counts["Testing"] += 1
        if any(keyword in token.text for keyword in deployment_keywords):
            category_counts["Deployment"] += 1
        if any(keyword in token.text for keyword in maintenance_keywords):
            category_counts["Maintenance"] += 1

    max_count = max(category_counts.values())
    if max_count == 0:
        task_type = "Other"
    else:
        task_type = [category for category,
                     count in category_counts.items() if count == max_count][0]
        
    # Check if task has bug fixes or code refactors
    task_has_bug_fixes = "true" if category_counts["Bug fixes"] != 0 else "false"
    task_has_codes_refactors = "true" if category_counts["Code Refactoring"] != 0 else "false"
    
    if(is_bug):
        task_type = "Bug fixes"
    if(is_ref):
        task_type = "Code Refactoring"
        
    return task_type, task_has_bug_fixes, task_has_codes_refactors

def get_has_bug_fixing(message, has_bugs):
    # Analyze the message with spaCy
    doc = nlp(message.lower())
    # Check for keywords and phrases related to each category
    for token in doc:
        if any(keyword in token.text for keyword in bug_keywords) or has_bugs:
            return True
        
    return False

def get_has_refactoring(message, has_refactor):
    # Analyze the message with spaCy
    doc = nlp(message.lower())
    # Check for keywords and phrases related to each category
    for token in doc:
        if any(keyword in token.text for keyword in refactoring_keywords) or has_refactor:
            return True
        
    return False

In [31]:
def categorize_time(timestamp):
    hour = datetime.datetime.strptime(timestamp, '%m/%d/%Y, %H:%M:%S').hour
    if 0 <= hour < 6:
        return 1  # 00:00-06:00AM
    elif 6 <= hour < 12:
        return 2  # 06:00-12:00
    elif 12 <= hour < 18:
        return 3  # 12:00-18:00
    else:
        return 4  # 18:00-23:59
    
def get_year(timestamp):
    year = datetime.datetime.strptime(timestamp, '%m/%d/%Y, %H:%M:%S').year
    return str(year)

def calculate_duration(authored_at, committed_at):
    """
    Calculates the duration between the authored_at and committed_at times.
    Returns a float representing the duration in hours.
    """
    authored_time = datetime.datetime.strptime(authored_at, '%m/%d/%Y, %H:%M:%S')
    committed_time = datetime.datetime.strptime(committed_at, '%m/%d/%Y, %H:%M:%S')
    duration = committed_time - authored_time
    return duration.total_seconds() / 3600


def convert_bool_to_int(bool_str):
    bool_str = str(bool_str).lower() 
    if bool_str.lower() == 'true':
        return 1
    elif bool_str.lower() == 'false':
        return 0
    else:
        raise ValueError("Input string must be 'true' or 'false'")


### Code quality calculator

In [26]:
def calculate_code_quality_probability(corrective_commits_count, non_corrective_commits_count):
    ccp = corrective_commits_count / (corrective_commits_count + non_corrective_commits_count)
    return 1 - ccp

The Corrective Commit Probability (CCP) code quality metric, which measures the probability that a given commit reflects corrective maintenance. A higher CCP value indicates a lower code quality, as it suggests that more bug fixes are required. Thus, we used a CCP compliment to get an estimated code quality value

In general, a high number of files affected or changed, a high number of lines added or deleted, or a high number of bug-fixing commits can indicate lower code quality. Smaller files, lower coupling, and the use of certain programming languages like JavaScript have been associated with higher code quality. 

### Sources


1. The Corrective Commit Probability Code Quality Metric, arxiv (https://arxiv.org/pdf/2007.10912.pdf) <br>
Abstract—We present a code quality metric, Corrective Commit Proba- bility (CCP), 
measuring the probability that a commit reflects corrective ...


2. [PDF] The Corrective Commit Probability Code Quality Metric, semanticscholar (https://www.semanticscholar.org/paper/The-Corrective-Commit-Probability-Code-Quality-Amit-Feitelson/1613bad06e6540f93d69100ee477628950f1799d) <br>
We present a code quality metric, Corrective Commit Probability (CCP), measuring 
the probability that a commit reflects corrective ...


3. Code and Commit Metrics of Developer Productivity, igor (https://www.igor.pro.br/publica/papers/EMSE2020.pdf) <br>
Based on commit information, we calculated the productivity of each developer by applying each 
commit-based metrics to their commits in the ...

### Code complexity calculator

In [32]:
def calculate_contribution_complexity(commit_shas):
    contribcompl = compute_contrib_compl(path_to_repo, [commit_shas])
    return contribcompl.value

### Sources
1. Automatically Assessing Complexity of
Contributions to Git Repositories, https://www.itu.dk/people/ropf/blog/assets/quatic2021_pfeiffer.pdf
   
   Source code: https://github.com/HelgeCPH/contribution-complexity

### Analyze the data using the appropriate analysis module

In [28]:
def analyze_tasks(json_obj, output, experience_path):
    # Initialize an empty list to store the tasks
    tasks = []

    # Define the regex pattern to match task IDs and messages
    regex = r"\[([A-Z]+-[0-9]+|[A-Z]+-[A-Z]+)\]\s*(\[(?:[A-Z]+-[0-9]+|[A-Z]+-[A-Z]+)\])?\s*(.*)"

    # Open experience csv
    experience_reader = read_csv_file(experience_path)

    # List of the header row (id, 20XX....)
    header = experience_reader[0]
    print('Timeline of the project: ', header[1:])

    
    # Store count of commits here
    corrective_commits_count = 0
    non_corrective_commits_count = 0

    # Iterate over each commit in the JSON json_obj
    for commit in json_obj["commits"]:
        # Commit hash
        commit_hash = commit["commit_id"]
        # Extract the author and committer
        authored_by = commit["authored_by"]["author_email"]
        committed_by = commit["committed_by"]["committer_email"]

        # Extract time information
        authored_at = commit["authored_at"]
        authored_year = get_year(authored_at)
        commited_at = commit["commited_at"]
        commited_year = get_year(commited_at)

        # Commit info
        commit_time = categorize_time(commited_at)

        # Wanted year
        a_year_index = header.index(authored_year)
        c_year_index = header.index(commited_year)

        # Experience in the company
        author_experience = 0
        for row in experience_reader[1:]:
            if (row[0] == commit["authored_by"]["author_email"]):
                author_experience = row[a_year_index]

        committer_experience = 0
        for row in experience_reader[1:]:
            if (row[0] == commit["committed_by"]["committer_email"]):
                committer_experience = row[c_year_index]

        # Extract the commit is_fix_related
        is_fix = commit["is_fix_related"]
        # Extract the commit is_bug_fixing
        is_bug = commit["is_bug_fixing"]
        # Extract the commit is_refactoring
        is_ref = commit["is_refactoring"]
        # Extract the commit total lines added in task
        added_lines = commit["total_lines_added_in_commit"]
        # Extract the commit total lines deleted in commit
        deleted_lines = commit["total_lines_deleted_in_commit"]
        # Extract the commit total files changed
        files_affected = commit["files_changed_total"]
        # Extract the commit total files changed
        files_changed = len(commit["changes"])
        # Extract the commit message
        message = commit["message"]

        # total code addition/deletions
        total_file_additions = 0
        total_file_deletions = 0

        for change in commit["changes"]:
            total_file_additions += change["file_additions"]
            total_file_deletions += change["file_deletions"]

        # Duration, code quality and code complexity is the focus here
        duration = calculate_duration(authored_at, commited_at)
        
        has_bug_fixing = get_has_bug_fixing(
            commit["message"], commit['is_bug_fixing'] or commit['is_fix_related'])
        
        corrective_commits_count += 1 if has_bug_fixing or is_fix or is_bug else 0 
        non_corrective_commits_count += 0 if has_bug_fixing or is_fix or is_bug else 1
        
        
        code_quality = calculate_code_quality_probability(corrective_commits_count, non_corrective_commits_count)
        contribution_complexity = calculate_contribution_complexity(commit["commit_id"])
        communication = authored_by != committed_by

        # Use regex to match the task ID and message
        match = re.match(regex, message)
        if match:
            # If the message matches the regex pattern, extract the task ID and message
            task_id = match.group(1)
            if match.group(2):
                task_id += "_" + re.sub(r"[\[\]]", "", match.group(2))
            task_message = match.group(3).strip()
            # Append the task to the list of tasks
            tasks.append({"id": commit_hash,
                          "message": task_message,
                          "is_fix_related": is_fix,
                          "is_bug_fixing": is_bug,
                          "is_refactoring": is_ref,
                          "total_lines_added_in_commit": added_lines,
                          "total_lines_deleted_in_commit": deleted_lines,
                          "total_files_affected": files_affected,
                          "total_files_changed": files_changed,
                          "author_email": authored_by,
                          "committer_email": committed_by,
                          "authored_at": authored_at,
                          "authored_year": authored_year,
                          "commited_at": commited_at,
                          "commited_year": commited_year,
                          "commit_time": commit_time,
                          "author_experience": author_experience,
                          "committer_experience": committer_experience,
                          "code_quality": code_quality,
                          "contribution_complexity": contribution_complexity,
                          "communication" : communication,
                          "duration": duration,
                          })

        else:
            # If the message doesn't match the regex pattern, use the entire message as the task message
            task_id = ""
            task_message = message
            # Append the task to the list of tasks
            tasks.append({"id": commit_hash,
                          "message": task_message,
                          "is_fix_related": is_fix,
                          "is_bug_fixing": is_bug,
                          "is_refactoring": is_ref,
                          "total_lines_added_in_commit": added_lines,
                          "total_lines_deleted_in_commit": deleted_lines,
                          "total_files_affected": files_affected,
                          "total_files_changed": files_changed,
                          "author_email": authored_by,
                          "committer_email": committed_by,
                          "authored_at": authored_at,
                          "authored_year": authored_year,
                          "commited_at": commited_at,
                          "commited_year": commited_year,
                          "commit_time": commit_time,
                          "author_experience": author_experience,
                          "committer_experience": committer_experience,
                          "code_quality": code_quality,
                          "contribution_complexity": contribution_complexity,
                          "communication" : communication,
                          "duration": duration,
                          })

    # write out the unique values to the output CSV file
    output_file_name = os.path.basename(output)

    # Write the list of tasks to a CSV file
    with open(output, mode='w') as f:
        writer = csv.writer(f)
        writer.writerow(["task_id",
                         "task_message",
                         "task_type",
                         "has_bug_fixing",
                         "has_code_refactoring",
                         'is_fix_related',
                         'is_bug_fixing',
                         'is_refactoring',
                         'total_lines_added_in_commit',
                         'total_lines_deleted_in_commit',
                         'total_files_affected',
                         'total_files_changed',
                         "author_email",
                         "committer_email",
                         "authored_at",
                         "authored_year",
                         "commited_at",
                         "commited_year",
                         "commit_time",
                         "author_experience",
                         "committer_experience",
                         "code_quality",
                         "contribution_complexity",
                         "communication",
                         "duration"
                         ])

        for task in tasks:
            writer.writerow([task["id"],
                             task["message"],
                             "",
                             "",
                             "",
                             task['is_fix_related'],
                             task['is_bug_fixing'],
                             task['is_refactoring'],
                             task['total_lines_added_in_commit'],
                             task['total_lines_deleted_in_commit'],
                             task['total_files_affected'],
                             task['total_files_changed'],
                             task["author_email"],
                             task["committer_email"],
                             task["authored_at"],
                             task["authored_year"],
                             task["commited_at"],
                             task["commited_year"],
                             task["commit_time"],
                             task["author_experience"],
                             task["committer_experience"],
                             task["code_quality"],
                             task["contribution_complexity"],
                             task["communication"],
                             task["duration"],
                             ])

    # Print the number of unique tasks found
    print(f"There are {len(tasks)} unique tasks.")
    return tasks


Call the function to create the file

In [29]:
json_analysis_results = analyze_tasks(
    json_obj, tasks_path, save_path + 'dev_experience.csv')

Timeline of the project:  ['2016', '2017', '2018', '2019']
There are 2770 unique tasks.


## Step 7

### Set the task informations by using the get_task_info function

In [36]:
def set_task_info(output, new_output):
    # Load the data from the CSV file
    df = pd.read_csv(output)
    
    
    # Apply the get_task_type function to the task_message column to generate the task_type column
    df[["task_type", "has_bug_fixing", "has_code_refactoring"]] = df.apply(lambda row: pd.Series(get_task_info(row["task_message"], row["is_fix_related"] or row["is_bug_fixing"], row["is_refactoring"])), axis=1)


    # Save the updated dataframe to a new CSV file
    output_file = os.path.join(
        os.path.dirname(output), new_output)
    df.to_csv(output_file, index=False)
    print(f"There are {len(df)} rows updated.")


Call the function to create the file

In [37]:
set_task_info(tasks_path, tasks_output_path)

There are 2770 rows updated.


## Step 8
### Add actual metrics calculation and Clean the data so that it can be used for causal estimations

#### Helper functions

In [38]:
def calculate_task_type(task_type):
    task_types = {
        "Feature requests or new feature": 1,
        "Bug fixes": 2,
        "Code Refactoring": 3,
        "Documentation": 4,
        "Testing": 5,
        "Deployment": 6,
        "Maintenance": 7,
        "Other": 8
    }

    return task_types.get(task_type, -1)


def convert_bool_to_int(bool_str):
    bool_str = str(bool_str)
    if bool_str.lower() == 'true':
        return 1
    elif bool_str.lower() == 'false':
        return 0
    else:
        raise ValueError("Input string must be 'true' or 'false'")


def get_index_of_dev(input_csv_file, email):
    csvreader = read_csv_file(input_csv_file)
    for row in csvreader[1:]:
        if (row[1] == email):
            return row[0]
    return -1

### Actual metrics calculation

Lizard is an open-source Cyclomatic Complexity Analyzer that is used here to compute a program's cyclomatic complexity.

Cyclomatic complexity is a software statistic that indicates a program's complexity by counting the number of linearly independent pathways across its source code. It is computed using the program's control-flow graph, where nodes correspond to groups of commands in the program and a directed edge connects two nodes if the second command might be executed shortly after the first command.

### Sources
1. Code Quality - Cyclomatic Complexity - Sticky Bits - Feabhas, https://blog.feabhas.com/2018/07/code-quality-cyclomatic-complexity/
Running Lizard, an open-source Cyclomatic Complexity Analyzer, ... the number of linearly independent paths through a program's source code. 

2. Cyclomatic complexity, https://en.wikipedia.org/wiki/Cyclomatic_complexity
Cyclomatic complexity is a software metric used to indicate the complexity of a program. It is a quantitative measure of the number of linearly independent paths ... The cyclomatic complexity of a section of source code is the number of ...

3. Cyclomatic Complexity, https://www.geeksforgeeks.org/cyclomatic-complexity/
It is a software metric used to indicate the complexity of a program. It is computed using the Control Flow Graph of the program. The nodes in ...


In [39]:
# Checkout to a specific commit hash in a Git repository.
def checkout_to_commit_hash(commit_hash):
    
    subprocess.run(['clear'], shell=True)

    # Run the Git checkout command
    subprocess.run(['git', 'checkout', commit_hash, '--f'])
    
    # Get metric
    total_nloc, avg_nloc, avg_ccn, avg_token, fun_cnt, warning_cnt, fun_rate, nloc_rate = analyze_lizard_metric()
    
    # Return metric
    return  total_nloc, avg_nloc, avg_ccn, avg_token, fun_cnt, warning_cnt, fun_rate, nloc_rate

# Calls the Lizard command line tool on a directory of source files and returns the code metrics as a map.
def analyze_lizard_metric():
    # Construct the Lizard command
    command = f'lizard .'
    result = subprocess.run(command, shell=True, capture_output=True, text=True)

    # Parse the output and extract the code metrics
    output_lines = result.stdout.split('\n')
    output_list = []
    for line in output_lines:
        line_list = line.split('\n')
        output_list.append(line_list)
        
    if(output_lines[-4].startswith('Total ')):
        last_line = output_lines[-2]
        values = last_line.split()
        metrics = [float(value) for value in values]
        # Create a map of the code metrics
        total_nloc = int(metrics[0])
        avg_nloc = float(metrics[1])
        avg_ccn = float(metrics[2])
        avg_token = float(metrics[3])
        fun_cnt = int(metrics[4])
        warning_cnt = int(metrics[5])
        fun_rate = float(metrics[6])
        nloc_rate = float(metrics[7])
        
        return total_nloc, avg_nloc, avg_ccn, avg_token, fun_cnt, warning_cnt, fun_rate, nloc_rate
    else:
        return "Error"
    

#### Reset path

In [40]:
# get the current working directory
current_dir = os.getcwd()

# change the current working directory to the parent directory
os.chdir(os.path.dirname(current_dir))

print(os.getcwd())
print("Reseted path")

/Users/blenassefa/Documents/CS/Thesis/thesis_experiments
Reseted path


In [38]:
def clean_csv_data(input_file_path, output_file_path, developer_csv):

    # open input and output files
    with open(input_file_path, 'r') as input_file:
        with open(output_file_path, 'w', newline='') as output_file:

        # define headers
            headers = [
                'task_type',
                'has_bug_fixing',
                'has_code_refactoring',
                'loc',
                'files_touched',
                'author',
                'committer',
                'authored_at',
                'commited_at',
                'commit_time_in_day',
                'author_experience',
                'committer_experience',
                'code_quality',
                'contribution_complexity',
                'communication',
                'total_nloc',
                'avg_nloc',
                'avg_ccn',
                'avg_token',
                'fun_cnt',
                'warning_cnt',
                'fun_rate',
                'nloc_rate',
                'duration',
                ]

        # create a CSV writer object and write headers to output file
            writer = csv.writer(output_file)
            writer.writerow(headers)

        # read each row from input file and process it
            reader = csv.reader(input_file)
            next(reader)  # skip header row

            cwd = os.getcwd()
            for row in reader:

            # extract values from specific columns
                task_type = calculate_task_type(row[2])
                has_bug_fixing = convert_bool_to_int(row[3])
                has_code_refactoring = convert_bool_to_int(row[4])
                loc = int(row[8]) + int(row[9])
                files_touched = int(row[10])
                author = get_index_of_dev(developer_csv, row[12])
                committer = get_index_of_dev(developer_csv, row[13])
                authored_at = datetime.datetime.strptime(row[14],
                        '%m/%d/%Y, %H:%M:%S')
                commited_at = datetime.datetime.strptime(row[16],
                        '%m/%d/%Y, %H:%M:%S')
                commit_time_in_day = int(row[18])
                author_experience = int(row[19])
                committer_experience = int(row[20])
                code_quality = float(row[21])
                contribution_complexity = float(row[22])
                communication = convert_bool_to_int(row[23])
                duration = float(row[24])
                  
                # Change the current working directory to the repository
                os.chdir(path_to_repo)

                (total_nloc, avg_nloc, avg_ccn, avg_token, fun_cnt, 
                 warning_cnt, fun_rate, nloc_rate) = checkout_to_commit_hash(row[0])
               
                
                # Change the working directory back to its original value
                os.chdir(cwd)

               # create a list of all values in the row
                row_data = [
                    task_type,
                    has_bug_fixing,
                    has_code_refactoring,
                    loc,
                    files_touched,
                    author,
                    committer,
                    authored_at,
                    commited_at,
                    commit_time_in_day,
                    author_experience,
                    committer_experience,
                    code_quality,
                    contribution_complexity,
                    communication,
                    total_nloc,
                    avg_nloc,
                    avg_ccn,
                    avg_token,
                    fun_cnt,
                    warning_cnt,
                    fun_rate,
                    nloc_rate,
                    duration,
                    ]

            # write the row to output file

                writer.writerow(row_data)
                
print("This marks complete. 🚀 🚀 🚀")

 Call the function to create the file

In [39]:
clean_csv_data(save_path + tasks_output_path, save_path +
               'clean_tasks.csv', save_path+"developers.csv")

Previous HEAD position was 68969e5 Remove this useless Views column in discussions;
HEAD is now at 003341b Initial commit
Previous HEAD position was 003341b Initial commit
HEAD is now at b2579a6 added .gitignore
Previous HEAD position was b2579a6 added .gitignore
HEAD is now at e854d43 A MVC base to get us started. Node: the model directory is empty because I haven't gotten around to playing with mongoose. Closes #17.


Previous HEAD position was e854d43 A MVC base to get us started. Node: the model directory is empty because I haven't gotten around to playing with mongoose. Closes #17.
HEAD is now at 8447890 Whoops, left out a comma.


Previous HEAD position was 8447890 Whoops, left out a comma.
HEAD is now at fcee690 Fixed missing module due to .gitignore


Previous HEAD position was fcee690 Fixed missing module due to .gitignore
HEAD is now at a0e738c Moved muExpress.js to a lib dir.


Previous HEAD position was a0e738c Moved muExpress.js to a lib dir.
HEAD is now at c289f3b Use the relative path of muExpress explicitly.


...

Previous HEAD position was 20a162a Some dep updates (#1689)
HEAD is now at 97e7998 npm depth omission (#1690)


Previous HEAD position was 97e7998 npm depth omission (#1690)
HEAD is now at a55d066 Some dep updates (#1691)


Previous HEAD position was a55d066 Some dep updates (#1691)
HEAD is now at 1926f7d Update Tampermonkey-for-Safari.md


Previous HEAD position was 1926f7d Update Tampermonkey-for-Safari.md
HEAD is now at 5cb52ef Some dep updates (#1692)


Previous HEAD position was 5cb52ef Some dep updates (#1692)
HEAD is now at b1e12f5 Pseudo breadcrumb Category column (#1693)


-- The End --